# 2章 Python数理最適化チュートリアル

### 2.1 連立一次方程式をPythonの数理最適化ライブラリで解く

In [1]:
"""
120x + 150y = 1440
x + y = 10
の単純な線形1次連立方程式を解くプログラム
"""

import pulp

problem = pulp.LpProblem('SLE', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve()

print(f"Status:{pulp.LpStatus[status]}")
print(f"x = {x.value()}, y = {y.value()}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/99bddab5212b46eb8ccf5d71979adb29-pulp.mps max branch printingOptions all solution /tmp/99bddab5212b46eb8ccf5d71979adb29-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 13 RHS
At line 16 BOUNDS
At line 20 ENDATA
Problem MODEL has 2 rows, 3 columns and 4 elements
Coin0008I MODEL read with 0 errors
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Status:Optimal
x = 2.0, y = 8.0


### 2.2 線形計画問題をPythonの数理最適化ライブラリで解く

解くべき最適化問題
   - max $n_p + 2n_q$
   - s.t.
      - $n_p, n_q >= 0$
      - $n_p+3n_q <= 30$
      - $2n_p+n_q <= 40$

In [8]:
"""
次のような線形計画問題を解くプログラム
   - max $n_p + 2n_q$
   - s.t.
      - $n_p, n_q >= 0$
      - $n_p+3n_q <= 30$
      - $2n_p+n_q <= 40$
"""

import pulp

problem = pulp.LpProblem("LP", pulp.LpMaximize)

np = pulp.LpVariable("n_p", cat="Continuous")
nq = pulp.LpVariable("n_q", cat="Continuous")

#制約条件の追加
problem += 1*np + 3*nq <= 30
problem += 2*np + 1*nq <= 40
problem += np >= 0
problem += nq >= 0

#目的関数
problem += np + 2*nq
#これらの書き方でも良い
# problem.setObjective(np + 2*nq)
# problem.objective = np + 2*nq

status = problem.solve()

print(f"status:{pulp.LpStatus[status]}")
print(f"n_p = {np.value()}, n_q = {nq.value()}, objective = {problem.objective.value()}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/36b18c48c70c4825a972990bd5201ed6-pulp.mps max branch printingOptions all solution /tmp/36b18c48c70c4825a972990bd5201ed6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 18 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj -0 Dual inf 2.999998 (2)
2  Obj 26
Optimal - objective value 26
After Postsolve, objective 26, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 26 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status:Optimal
n_p = 18.0, n_q = 4.0, object

#### 最適化問題の種類
この辺は詳細な書籍を読んだ方が正確な知識は得られそう。
  - 線形計画問題：変数が実数値をとる問題
  - 整数計画問題：変数が整数値のみを取る問題
  - 混合整数計画問題：変数が整数値を取るものと実数値を取るものがある問題
  - 0-1整数計画問題：変数が0か1で表せる問題
  - 凸二次計画問題：目的関数に凸な二次関数（係数行列が半正値対称行列）が現れる問題

### 2.3 規模の大きな数理最適化問題をPythonライブラリを使って解く

解くべき最適化問題
   - max $c^{T}x =  \left(
\begin{matrix} 
3  \\ 
4  \\
4  \\
5  \\
\end{matrix} 
\right) 
\left(
\begin{matrix} 
x_{p_1} & x_{p_2} & x_{p_3} & x_{p_4} \\ 
\end{matrix} 
\right) = 3x_{p_1} + 4x_{p_2} + 4x_{p_3} + 5x_{p_4}$
   - s.t.
      - $x_{p_1}, x_{p_2}, x_{p_3}, x_{p_4} >= 0$
      - $\left(\begin{matrix} r_{p_1, m_1} & r_{p_2, m_1} & r_{p_3, m_1} & \ \ r_{p_4, m_1}　\\ 
r_{p_1, m_2} & r_{p_2, m_2} & r_{p_3, m_2} & r_{p_4, m_2}  \\
r_{p_1, m_3} & r_{p_2, m_3} & r_{p_3, m_3} & r_{p_4, m_3}  
\end{matrix} 
\right)
      \left(\begin{matrix} \ \ x_{p_1}　\\ 
x_{p_2}  \\
x_{p_3}  \\
x_{p_4}  \\
\end{matrix} 
\right) <= \left(
\begin{matrix} 
stock_{m_1}  \\ 
stock_{m_2}  \\
stock_{m_3}  \\
\end{matrix} 
\right) $
      - $\left(\begin{matrix} r_{p_1, m_1} & r_{p_2, m_1} & r_{p_3, m_1} & \ \ r_{p_4, m_1}　\\ 
r_{p_1, m_2} & r_{p_2, m_2} & r_{p_3, m_2} & r_{p_4, m_2}  \\
r_{p_1, m_3} & r_{p_2, m_3} & r_{p_3, m_3} & r_{p_4, m_3}  
\end{matrix} 
\right) = \left(\begin{matrix} 2 & 3 & 0 & \ \ 2　\\ 
0 & 2 & 2 & 2  \\
1 & 0 & 2 & 2  
\end{matrix} 
\right)$
      - $\left(
\begin{matrix} 
stock_{m_1}  \\ 
stock_{m_2}  \\
stock_{m_3}  \\
\end{matrix} 
\right) = \left(
\begin{matrix} 
35  \\ 
22  \\
27  \\
\end{matrix} 
\right)$

In [9]:
import pandas as pd
import pulp

##### 在庫データの確認

In [10]:
stock_df = pd.read_csv("./data/stocks.csv")
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


##### 製品の生産に必要な原料の量データの確認

In [12]:
require_df = pd.read_csv("./data/requires.csv")
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


##### 製品を生産した場合の利得データの確認

In [13]:
gain_df = pd.read_csv("./data/gains.csv")
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


#### 必要変数のリストと定数データの作成

In [15]:
Product_list = gain_df["p"].tolist()
Product_list

['p1', 'p2', 'p3', 'p4']

In [18]:
Material_list = stock_df["m"].tolist()
Material_list

['m1', 'm2', 'm3']

In [21]:
stock_dict = stock_df.set_index("m").to_dict()["stock"]
stock_dict

{'m1': 35, 'm2': 22, 'm3': 27}

In [25]:
require_dict = require_df.set_index(["p", "m"]).to_dict()["require"]
require_dict

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [26]:
gain_dict = gain_df.set_index("p").to_dict()["gain"]
gain_dict

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

#### 線形計画問題の定義

In [27]:
problem = pulp.LpProblem("LP-tutorial-2.3", pulp.LpMaximize)

#### 変数の定義

In [39]:
#製品pの生産量
x = pulp.LpVariable.dicts("x", Product_list, cat="Continuous")
print(x)

{'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}


#### 制約式の定義

In [40]:
for p in Product_list:
    problem += x[p] >= 0

In [43]:
for m in Material_list:
    # sumよりもlpSumの方が高速
    problem += pulp.lpSum([require_dict[p,m] * x[p] for p in Product_list]) <= stock_dict[m]

#### 目的関数の設定

In [44]:
problem.objective = pulp.lpSum([gain_dict[p] * x[p] for p in Product_list])

#### 問題設定の確認

In [48]:
print(problem)

LP-tutorial-2.3:
MAXIMIZE
3*x_p1 + 4*x_p2 + 4*x_p3 + 5*x_p4 + 0
SUBJECT TO
_C1: x_p1 >= 0

_C2: x_p2 >= 0

_C3: x_p3 >= 0

_C4: x_p4 >= 0

_C5: 2 x_p1 + 3 x_p2 + 2 x_p4 <= 35

_C6: 2 x_p2 + 2 x_p3 + 2 x_p4 <= 22

_C7: x_p1 + 2 x_p3 + 2 x_p4 <= 27

VARIABLES
x_p1 free Continuous
x_p2 free Continuous
x_p3 free Continuous
x_p4 free Continuous



#### 最適化問題の求解

In [45]:
status = problem.solve()
print(f"status: {pulp.LpStatus[status]}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/e3684899c0d54774beb666b24eaabceb-pulp.mps max branch printingOptions all solution /tmp/e3684899c0d54774beb666b24eaabceb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status: Opt

#### 最適解と目的関数値の表示

In [47]:
for p in Product_list:
    print(f"Product:{p}, Number:{x[p].value()}")
    
print(f"Objective:{problem.objective.value()}")

Product:p1, Number:12.142857
Product:p2, Number:3.5714286
Product:p3, Number:7.4285714
Product:p4, Number:0.0
Objective:80.42857099999999


#### 2.4 整数最適化問題への変換

先の最適化問題に変数が整数であるという制約条件を加えて最適化してみる
   - max $c^{T}x =  \left(
\begin{matrix} 
3  \\ 
4  \\
4  \\
5  \\
\end{matrix} 
\right) 
\left(
\begin{matrix} 
x_{p_1} & x_{p_2} & x_{p_3} & x_{p_4} \\ 
\end{matrix} 
\right) = 3x_{p_1} + 4x_{p_2} + 4x_{p_3} + 5x_{p_4}$
   - s.t.
      - $x_{p_1}, x_{p_2}, x_{p_3}, x_{p_4} \in Z$
      - $x_{p_1}, x_{p_2}, x_{p_3}, x_{p_4} >= 0$
      - $\left(\begin{matrix} r_{p_1, m_1} & r_{p_2, m_1} & r_{p_3, m_1} & \ \ r_{p_4, m_1}　\\ 
r_{p_1, m_2} & r_{p_2, m_2} & r_{p_3, m_2} & r_{p_4, m_2}  \\
r_{p_1, m_3} & r_{p_2, m_3} & r_{p_3, m_3} & r_{p_4, m_3}  
\end{matrix} 
\right)
      \left(\begin{matrix} \ \ x_{p_1}　\\ 
x_{p_2}  \\
x_{p_3}  \\
x_{p_4}  \\
\end{matrix} 
\right) <= \left(
\begin{matrix} 
stock_{m_1}  \\ 
stock_{m_2}  \\
stock_{m_3}  \\
\end{matrix} 
\right) $
      - $\left(\begin{matrix} r_{p_1, m_1} & r_{p_2, m_1} & r_{p_3, m_1} & \ \ r_{p_4, m_1}　\\ 
r_{p_1, m_2} & r_{p_2, m_2} & r_{p_3, m_2} & r_{p_4, m_2}  \\
r_{p_1, m_3} & r_{p_2, m_3} & r_{p_3, m_3} & r_{p_4, m_3}  
\end{matrix} 
\right) = \left(\begin{matrix} 2 & 3 & 0 & \ \ 2　\\ 
0 & 2 & 2 & 2  \\
1 & 0 & 2 & 2  
\end{matrix} 
\right)$
      - $\left(
\begin{matrix} 
stock_{m_1}  \\ 
stock_{m_2}  \\
stock_{m_3}  \\
\end{matrix} 
\right) = \left(
\begin{matrix} 
35  \\ 
22  \\
27  \\
\end{matrix} 
\right)$

In [49]:
#変数準備
Product_list = gain_df["p"].tolist()
Material_list = stock_df["m"].tolist()

stock_dict = stock_df.set_index("m").to_dict()["stock"]
require_dict = require_df.set_index(["p", "m"]).to_dict()["require"]
gain_dict = gain_df.set_index("p").to_dict()["gain"]

#最適化問題の定式化
problem = pulp.LpProblem("IP-tutorial-2.4", pulp.LpMaximize)

#変数（製品pの生産量）の定義
x = pulp.LpVariable.dicts("x", Product_list, cat="Integer")

# 制約条件の定式化
for p in Product_list:
    problem += x[p] >= 0
for m in Material_list:
    # sumよりもlpSumの方が高速
    problem += pulp.lpSum([require_dict[p,m] * x[p] for p in Product_list]) <= stock_dict[m]
    
problem.objective = pulp.lpSum([gain_dict[p] * x[p] for p in Product_list])

print(problem)

IP-tutorial-2.4:
MAXIMIZE
3*x_p1 + 4*x_p2 + 4*x_p3 + 5*x_p4 + 0
SUBJECT TO
_C1: x_p1 >= 0

_C2: x_p2 >= 0

_C3: x_p3 >= 0

_C4: x_p4 >= 0

_C5: 2 x_p1 + 3 x_p2 + 2 x_p4 <= 35

_C6: 2 x_p2 + 2 x_p3 + 2 x_p4 <= 22

_C7: x_p1 + 2 x_p3 + 2 x_p4 <= 27

VARIABLES
x_p1 free Integer
x_p2 free Integer
x_p3 free Integer
x_p4 free Integer



#### 求解

In [50]:
status = problem.solve()
print(f"status: {pulp.LpStatus[status]}")

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /home/ren-ito/anaconda3/envs/numerical-optimizaion-env/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/56570c8215084898be4eae577bcd59e4-pulp.mps max branch printingOptions all solution /tmp/56570c8215084898be4eae577bcd59e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -76 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0038I Full problem 3 rows 4 columns, reduced to 3 rows 3 columns
Cbc0012I Integer solution of -79 found by RINS after 0 iteration

#### 最適解と目的関数の表示

In [51]:
for p in Product_list:
    print(f"Product:{p}, Number:{x[p].value()}")
    
print(f"Objective:{problem.objective.value()}")

Product:p1, Number:13.0
Product:p2, Number:3.0
Product:p3, Number:7.0
Product:p4, Number:-0.0
Objective:79.0
